# Stepper dancer

In [1]:
import serial
import time
import ctypes
import perlin_noise
import datetime
import random

import pyaudio
import math

In [40]:
# Set up serial communications
port = "/dev/cu.usbserial-1420"
baud = 115200
ser = serial.Serial(port, baud)
time.sleep(2)

In [41]:
# Create an instance of PyAudio
audio = pyaudio.PyAudio()

# Open a stream to capture audio from the microphone
stream = audio.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=44100,
                    input=True,
                    frames_per_buffer=1024)

In [4]:
def serialize(motor, acceleration, speed, revolutions):
    motor_b = int(motor).to_bytes()
    acceleration_b = acceleration.to_bytes(2, signed = True)
    speed_b = speed.to_bytes(2, signed = True)
    revolutions_b = int(revolutions * 16.0 * 200.0).to_bytes(2, signed = True)
    print(( motor_b + acceleration_b + speed_b + revolutions_b).hex(' ', 2) )
    return motor_b + acceleration_b + speed_b + revolutions_b

def serialize_float(motor, acceleration, speed, revolutions):
    acceleration_32 = ctypes.c_float(acceleration)
    speed_32 = ctypes.c_float(speed)
    revolutions_32 = ctypes.c_float(revolutions)
    
    motor_b = int(motor).to_bytes()
    acceleration_b = bytes(acceleration_32)
    speed_b = bytes(speed_32)
    revolutions_b = bytes(revolutions_32)
    return (motor_b + acceleration_b + speed_b + revolutions_b)

def get_rms(frames):
    count = len(frames)/2.0
    sum_squares = 0.0
    sum_samples = 0.0

    for i in range(0, len(frames), 2):
        # Unpack the frame (2 bytes) into a signed integer
        n = int.from_bytes(frames[i:i+2], byteorder='little', signed=True)
        sum_squares += n ** 2
        sum_samples += n

    rms = math.sqrt(sum_squares / count)
    dc = sum_samples / count
    return rms
    return (rms, dc)

In [ ]:
acceleration = 1000.0
moving_average = 0.0
moving_average_num = 10.0

while True:
    # Read audio data from the stream
    frames = b''
    for i in range(4):
        data = stream.read(1024, exception_on_overflow = False)
        frames += data

    # print("Finished recording.")
    rms_value = get_rms(frames)
    moving_average = (moving_average * moving_average_num + rms_value) / (moving_average_num + 1)
    # print(f"values: {rms_value}")

    myspeed = moving_average / 10000.0
    print(myspeed, end='\r')
    ser.write( serialize_float(motor = 0, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    # print(ser.read_all(), end="")
    ser.write( serialize_float(motor = 1, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    # print(ser.read_all(), end="")
    ser.write( serialize_float(motor = 2, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    # print(ser.read_all(), end="")
    ser.write( serialize_float(motor = 3, acceleration = acceleration, speed = myspeed, revolutions = 1 ) )
    # print(ser.read_all())


In [43]:
stream.stop_stream()
stream.close()
audio.terminate()